Code that is not currently used, but might be useful to have around in case want to remember how I once did something.

#### Plotting transparent backgrounds

In [ ]:
targeted_phenos = rank_top_phenos('present',20,1,1)

# Standardize yield_mean & yield_disp into between 0 & 1
yield_mean_norm = (
    yield_mean-yield_mean.min())/(yield_mean.max()-yield_mean.min())
yield_disp_norm = (
    yield_disp-yield_disp.min())/(yield_disp.max()-yield_disp.min())

# visulization
with plt.rc_context({'axes.edgecolor':'whitesmoke', 
                     'xtick.color':'whitesmoke',
                     'ytick.color':'whitesmoke',
                     'figure.facecolor':'white'}):

    fig = plt.figure(figsize=(9,5))
    ax = fig.add_subplot()
    sc = ax.scatter(yield_mean_norm, yield_disp_norm, 
                    marker='o', facecolor='none', edgecolor='whitesmoke', 
                    s=100, alpha=0.5)
    ax.scatter(yield_mean_norm[targeted_phenos], yield_disp_norm[targeted_phenos],
               c=np.arange(len(targeted_phenos)), cmap=PurpOr_6.mpl_colormap.reversed(), 
               s=100, alpha=0.9)
    for i in np.arange(100):
        ax.annotate(i, (yield_mean_norm[i], yield_disp_norm[i]), size=8, c='lightgrey')

    ax.set_xlabel('yield mean', color='white', fontweight='light', size=14)
    ax.set_ylabel('dispersion index', color='white', fontweight='light', size=14)
    ax.set_title('Yield mean and disparsion - averaged over all sites', 
                 color='white', fontweight='light', size=15)

plt.savefig('/home/disk/eos8/ach315/upscale/figs/scatter_yield_disp_standardize_top_performance_transp.png', 
            transparent=True, format='png', dpi=800)

In [ ]:
top_phenos = rank_top_phenos('present', 20, 1, 1)
n_phenos_toplot = 20
fig_w=9
fig_h=6

with plt.rc_context({'axes.edgecolor':'whitesmoke', 
                     'xtick.color':'whitesmoke',
                     'ytick.color':'whitesmoke',
                     'figure.facecolor':'white'}):

    problem, param_values = params_sample('present', 10)
    param_range = dict(zip(problem['names'], problem['bounds']))
    params = problem['names']
    df_params_fold = pd.DataFrame(columns=params)

    # Normalize parameter values
    if n_phenos_toplot == -1:
        n_phenos_toplot = len(top_phenos)

    if n_phenos_toplot > len(top_phenos):
        n_phenos_toplot = len(top_phenos)

    df_highperformance = df_params.iloc[top_phenos[:n_phenos_toplot], :-1]

    for param in params:
        df_params_fold[param] = fold(df_highperformance[param],
                                     param_range[param][0],
                                     param_range[param][1])

    # Visualize
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))

    ax.imshow(df_params_fold.transpose(), cmap=PuOr_7.mpl_colormap)
    ax.set_xticks(np.arange(df_highperformance.shape[0]))
    ax.set_yticks(np.arange(df_highperformance.shape[1]))
    ax.set_xticklabels(list(df_highperformance.index),
                       size=10, fontweight='light')
    ax.set_yticklabels(['g1', 'Vcmax', 'Jmax', 'phyf', 
                        'sg', 'leaf', 'ltar', 'len',
                        'laf', 'pdate', 'pop'],
                       size=10, fontweight='light')
#    ax.set_yticklabels(list(df_highperformance.columns),
#                       size=10, fontweight='light')

    for top_pheno in np.arange(n_phenos_toplot):
        for param in range(len(params)):
            ax.text(top_pheno, param,
                    df_params.transpose().loc[params[param],
                                              top_phenos[top_pheno]],
                    ha='center', color='grey', size=7)
    fig.subplots_adjust(left=0.15)
    
plt.savefig('/home/disk/eos8/ach315/upscale/figs/heatmap_params_ranktop_20_transp.png',
            transparent=True, format='png', dpi=800)

#### Density estimation & contour plots

In [ ]:
def density_estimation(m1, m2):
    xmin = min(m1)
    xmax = max(m1)
    ymin = min(m2)
    ymax = max(m2)
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = stats.gaussian_kde(values)                                                                 
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

In [ ]:
# rank phenos
phenos_ranked = rank_top_phenos('present', 100, 1, 1)

# put together params info
df_params_s = df_params.iloc[:,:-1][:100]
df_params_sub = df_params_s.iloc[phenos_ranked]

xs = df_params_sub['juv_leaves']
ys = df_params_sub['rmax_ltar']

xs_list = []
for item, x in enumerate(xs):
    if item < 50:
        xs_amplified = [[x]*(100-item)]
    else:
        xs_amplified = [[x]*(1)]
    
    xs_list.append(xs_amplified)

ys_list = []
for item, y in enumerate(ys):
    if item < 50:
        ys_amplified = [[y]*(100-item)]
    else:
        ys_amplified = [[y]*(1)]
    
    ys_list.append(ys_amplified)

    
xs_flat = list(itertools.chain(*xs_list))
ys_flat = list(itertools.chain(*ys_list))
xs_flat2 = list(itertools.chain(*xs_flat))
ys_flat2 = list(itertools.chain(*ys_flat))

X, Y, Z = density_estimation(xs_flat2, ys_flat2)
plt.contour(X, Y, Z, 
            cmap=PurpOr_6.mpl_colormap)


In [ ]:
# rank phenos
phenos_ranked = rank_top_phenos('f2100', 100, 1, 1)

# put together params info
df_params_s = df_params.iloc[:,:-1][:100]
df_params_sub = df_params_s.iloc[phenos_ranked]

xs = df_params_sub['juv_leaves']
ys = df_params_sub['rmax_ltar']

xs_list = []
for item, x in enumerate(xs):
    if item < 50:
        xs_amplified = [[x]*(100-item)]
    else:
        xs_amplified = [[x]*(1)]
    
    xs_list.append(xs_amplified)

ys_list = []
for item, y in enumerate(ys):
    if item < 50:
        ys_amplified = [[y]*(100-item)]
    else:
        ys_amplified = [[y]*(1)]
    
    ys_list.append(ys_amplified)

    
xs_flat = list(itertools.chain(*xs_list))
ys_flat = list(itertools.chain(*ys_list))
xs_flat2 = list(itertools.chain(*xs_flat))
ys_flat2 = list(itertools.chain(*ys_flat))

X, Y, Z = density_estimation(xs_flat2, ys_flat2)
 
plt.contour(X, Y, Z, 
            cmap=PurpOr_6.mpl_colormap)
